In [2]:
"""
Data Preprocessing
"""

import os
import re
import pandas as pd

# read train data
file_dir = '/Users/19501/OneDrive/Desktop/SML/ASS1/data'
file_name = "train.txt"

def parse(data:list)->list: 
    # Python Notation: https://www.python.org/dev/peps/pep-3107/
    """
    Prase each line of the data
    Return a nested list, such as [[id1, id2, id2,...],...]
    """
    parsed_data = []
    for line in data:
        # remove \n at the end of each line
        line = re.sub(r"(?<=\d)\n", "", line)
        # split IDs by \t
        pattern = re.compile("(?<=\d)\t(?=\d)")
        line = re.split(pattern, line)
        parsed_data.append(line)
    return parsed_data

with open(os.path.join(file_dir, file_name)) as f:
    train_set = f.readlines()
    train_set = parse(train_set)
    
# read sub_train data
test_name = "sub_train.txt"
with open(os.path.join(file_dir, test_name)) as f:
    test_set = f.readlines()
    test_set = parse(test_set)
    test_set.pop(0)
# turn ID into int
for i in range(0, len(test_set)):
    test_set[i][0] = int(test_set[i][0])

train_dic = {}
for item in train_set:
    train_dic[item[0]] = item[1:]

In [3]:
from collections import defaultdict


def getSigmaOut(node:str, graph:dict) -> set:
    SigmaOut = set()
    try:
        for item in graph[node]:
            SigmaOut.add(item)
        return SigmaOut
    except:
        return SigmaOut

    
# construct Sigma Out dic
SigmaOut = defaultdict(set)

for kw in train_dic:
    SigmaOut[kw] = getSigmaOut(kw, train_dic)

# construct Sigma In dic
SigmaIn = defaultdict(set)

for kw in train_dic:
    for followee in train_dic[kw]:
        if SigmaIn[followee]:
            SigmaIn[followee].add(kw)
        else:
            SigmaIn[followee] = {kw}

def getSigma(node:str) -> set:
    return SigmaIn[node].union(SigmaOut[node])

In [4]:

def jaccard1(a:str, b:str) -> float:
    SigmaA = getSigma(a)
    SigmaB = getSigma(b)
    return float(len(SigmaA.intersection(SigmaB)) / len(SigmaA.union(SigmaB)))


def jaccard2(a:str, b:str) -> float:
    score = 0
    SigmaOutA = SigmaOut[a]
    if SigmaOutA:
        for node in SigmaOutA:
            score += jaccard1(b, node)
        return float(score/len(SigmaOutA))
    else:
        return 0

def jaccard3(a:str, b:str) -> float:
    score = 0
    SigmaInB = getSigmaIn[b]
    if SigmaInB:
        for node in SigmaInB:
            score += jaccard1(a, node)
        return float(score/len(SigmaInB))
    else:
        return 0

def cosine1(a:str, b:str) -> float:
    SigmaA = getSigma(a)
    SigmaB = getSigma(b)
    return float(len(SigmaA.intersection(SigmaB)) / (len(SigmaA)*len(SigmaB)))


def cosine2(a:str, b:str) -> float:
    score = 0
    SigmaOutA = getSigmaOut[a]
    if SigmaOutA:
        for node in SigmaOutA:
            score += cosine1(b, node)
        return float(score/len(SigmaOutA))
    else:
        return 0

def cosine3(a:str, b:str) -> float:
    score = 0
    SigmaInB = getSigmaIn[b]
    if SigmaOutB:
        for node in SigmaInB:
            score += cosine1(a, node)
        return float(score/len(SigmaInB))
    else:
        return 0

In [ ]:
import multiprocessing as mp
import time
import copy as cp

if __name__ == '__main__':
    def getPro(record:list) -> list:
        print(2)
        node = [record[0], jaccard2(record[1], record[-1]),jaccard3(record[1], record[-1]),
           cosine2(record[1], record[-1]), cosine3(record[1], record[-1])]
        print(node)
        return node
    # construct the # of pools corresponding to the cpu_count in ur PC
    pool = mp.Pool(mp.cpu_count())
    print(mp.cpu_count())
    startTime = time.time()
    
    sub_train_data = pool.map(getPro, test_set[:2])
    pool.close()
    pool.join()
    
    endTime = time.time()
    print("Total time:" + (endTime - startTime).__str__())

    title = ["Id", "J2", "J3", "C2", "C3"]
    sub_train_data = pd.DataFrame(columns = title, data = sub_train_data)
    sub_train_data.to_csv('sub_train_data.csv',encoding='utf-8')

8
